# WCA Results - Partial Senior Rankings

Created by Michael George (AKA Logiqx)

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [1]:
from EventsLib import *

import os
import csv
import json

## Read WCA Lookups

Read events, countries and continents from CSVs

In [2]:
class WcaEvent:
    def __init__(self, fields):
        """Initialisise the event"""
        
        self.id = fields[0]
        self.name = fields[1]
        self.format = fields[2]
        self.rankings = []

def readWcaEvents():
    """Read events from CSV into memory"""

    events = []
    
    fn = os.path.join('..', 'data', 'public', 'wca_lookup_events.csv')
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)

        for inputRow in csvReader:             
            events.append(WcaEvent(inputRow))
            
    return events

wcaEvents = readWcaEvents()

In [3]:
class Continent:
    def __init__(self, fields):
        """Initialisise the continent"""
        self.id = fields[0]
        self.name = fields[1]

def readContinents():
    """Read continents from CSV into memory"""

    continents = []

    fn = os.path.join('..', 'data', 'public', 'wca_lookup_continents.csv')
    with open(fn, 'r') as f:

        csvReader = csv.reader(f)
        for inputRow in csvReader:             
            continents.append(Continent(inputRow))

    return continents

continents = readContinents()

continentsDict = {}
for continent in continents:
    continentsDict[continent.id] = continent

In [4]:
class Country:
    def __init__(self, fields):
        """Initialisise the country"""
        self.id = fields[3]
        self.name = fields[1]
        self.continent = fields[2]
        self.persons = []

def readCountries():
    """Read countries from CSV into memory"""

    countries = []

    fn = os.path.join('..', 'data', 'public', 'wca_lookup_countries.csv')
    with open(fn, 'r') as f:

        csvReader = csv.reader(f)
        for inputRow in csvReader:             
            country = Country(inputRow)
            country.continent = continentsDict[country.continent].name
            countries.append(country)

    countries = sorted(countries, key=lambda country: country.id)

    return countries

countries = readCountries()

countriesMap = {}
for country in countries:
    countriesMap[country.name] = country.id

countriesDict = {}
for country in countries:
    countriesDict[country.id] = country

## Read Partial Results from CSV

Read event data from CSV into memory, prior to processing

In [5]:
class Person:
    
    def __init__(self, fields):
        """Initialisise the person"""
        
        self.id = fields[0]
        self.name = fields[1]
        self.country = fields[2]
        self.username = fields[3]
        self.usernum = int(fields[4])
        self.age = int(fields[5])
        if fields[6].lower() == 'y':
            self.name = '* ' + self.name + ' *'
        self.events = []


    def getWcaLink(self, eventId = None):
        if (eventId):
            link = '<a href="https://www.worldcubeassociation.org/persons/%s#%s">%s</a>' % (self.id, eventId, self.name)
        else:
            link = '<a href="https://www.worldcubeassociation.org/persons/%s">%s</a>' % (self.id, self.name)

        return link


    def getSpeedsolvingLink(self):
        if self.usernum > 0:
            profile = '%s.%d' % (self.username.replace(' ', '-').replace('.', '-').lower(), self.usernum)
            link = '<a href="https://www.speedsolving.com/members/%s">%s</a>' % (profile, self.username)
        else:
            link = ''

        return link

def readPersons():
    """Read persons from CSV into memory"""

    persons = []

    fn = os.path.join('..', 'data', 'public', 'known_senior_details.csv')
    with open(fn, 'r') as f:

        csvReader = csv.reader(f)
        for inputRow in csvReader:
            person = Person(inputRow)
            person.country = countriesMap[person.country]
            if person.usernum <= 0:
                del(person.username)
                del(person.usernum)
            persons.append(person)
            
    persons = sorted(persons, key=lambda person: person.id)

    for person in persons:
        countriesDict[person.country].persons.append(person.id)

    return persons

persons = readPersons()

personsDict = {}
for person in persons:
    personsDict[person.id] = person

In [6]:
class PartialResults:
    
    def __init__(self, age):
        """Initialisise the partial results"""
        
        self.event = None
        self.persons = {}
        self.ids = []
        self.results = {}
        self.age = age


    def readPersons(self, basename):
        """Read seniors from CSV into memory"""
        
        self.persons = {}

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', basename + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for line in csvReader:
                person = Person(line)

                if person.age >= self.age and person.id not in self.persons:
                    self.persons[person.id] = person
                    self.ids.append(person.id)


    def listPersons(self):
        """List seniors from memory"""
        
        html = '<details id="persons">\n'
        html += '  <summary>%s</summary>\n' % 'Known Over %ds' % self.age
        html += '  <table>\n'
        html += '    <tr>'
        for field in ['Person', 'Speedsolving.com']:
            html += '<td><b>%s</b></td>' % field
        html += '</tr>\n'
            
        for id in self.ids:
            person = self.persons[id]
            ageSuffix = ', %d+' % int(person.age) if int(person.age) > self.age else ''

            html += '    <tr>'
            html += '<td>%s, %s%s</td>' % (person.getWcaLink(), person.country, ageSuffix)
            html += '<td>%s</td>' % person.getSpeedsolvingLink()
            html += '</tr>\n'

        html += '  </table>\n'
        html += '</details>\n\n'
        
        return html
            
        
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.event = event
        self.results = []
        
        if event[0] == '333fm' and 'single' in basename:
            self.fmSingle = True
        else:
            self.fmSingle = False

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', basename, event[0] + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            personIds = {}
            for inputRow in csvReader:             
                personAge = inputRow[2]
                wcaId = inputRow[0]

                if int(personAge) >= self.age:
                    if wcaId not in personIds:
                        self.results.append(inputRow)
                        personIds[wcaId] = True

                    if event[0] not in personsDict[wcaId].events:
                        personsDict[wcaId].events.append(event[0])


    def listResults(self, section):
        """List seniors from memory"""
        
        html = ''
        rankings = \
        {
            'type': 'average' if section == 'avg' else 'single',
            'age': self.age,
            'ranks': []
        }
        
        if len(self.results) > 0:
            html = '<details id="%s_%s">\n' % (self.event[0], section)
            html += '  <summary>%s</summary>\n' % self.event[1]
            html += '  <table>\n'
            html += '    <tr>'
            for field in ['Rank', 'Person', 'Result']:
                html += '<td><b>%s</b></td>' % field
            html += '</tr>\n'

            count = rank = 1
            prevResult = None

            for result in self.results:
                person = self.persons[result[0]]
                ageSuffix = ', %d+' % int(result[2]) if int(result[2]) > self.age else ''

                if self.fmSingle:
                    formattedResult = formatResult(self.event, int(result[1]) * 100, showFractions = False)
                else:
                    formattedResult = formatResult(self.event, result[1], showFractions = True)

                if formattedResult != prevResult:
                    rank = count

                html += '    <tr>'
                html += '<td style="text-align:center">%s</td>' % (rank if formattedResult != prevResult else '')
                html += '<td>%s, %s%s</td>' % (person.getWcaLink(self.event[0]), person.country, ageSuffix)
                html += '<td style="text-align:right">%s</td>' % formattedResult
                html += '</tr>\n'
                
                best = \
                    {
                        'rank': rank,
                        'id': person.id,
                        'best': formattedResult
                    }
                if int(result[2]) > self.age:
                    best['age'] = int(result[2])
                rankings['ranks'].append(best)

                prevResult = formattedResult
                count += 1

            html += '  </table>\n'
            html += '</details>\n\n'
        
        for wcaEvent in wcaEvents:
            if wcaEvent.id == self.event[0] and len(rankings['ranks']) > 0:
                wcaEvent.rankings.append(rankings)
                break
        
        return html

## Analyse Events

Process the events one-by-one

In [7]:
import datetime

def runReport(age):
    
    fn = 'Partial_Rankings_%d.md' % age
    with open(os.path.join('..', 'templates', fn), 'r') as f:
        html = ''.join(f.readlines())

    html += 'Last refreshed: ' + refreshed + ' (UTC)\n\n'

    partialResults = PartialResults(age)

    html += '<h2 id="competitors">%s</h2>\n\n' % 'Official Competitors'
    fn = 'known_senior_details'
    partialResults.readPersons(fn)
    html += partialResults.listPersons()
    
    ids = partialResults.ids
    ids.sort()
    idsCsv = ','.join(id for id in ids)
    html = html.replace("wcaids=", "wcaids=" + idsCsv)

    html += '<h2 id="averages">%s</h2>\n\n' % 'Official Averages'
    for event in events:
        if event[3] != 'multi':
            fn = 'known_senior_averages'
            partialResults.readResults(fn, event)
            html += partialResults.listResults("avg")

    html += '<h2 id="singles">%s</h2>\n\n' % 'Official Singles'
    for event in events:
        fn = 'known_senior_singles'
        partialResults.readResults(fn, event)
        html += partialResults.listResults("best")

    fn = 'Partial_Rankings_%d.md' % age
    with open(os.path.join('..', 'docs', fn), 'w') as f:
        f.write(html)

refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')

for wcaEvent in wcaEvents:
    wcaEvent.rankings = []

runReport(age = 40)
runReport(age = 50)
runReport(age = 60)
runReport(age = 70)
runReport(age = 80)

for i in range(len(persons) - 1, -1, -1):
    if len(persons[i].events) == 0:
        countriesDict[persons[i].country].persons.remove(persons[i].id)
        del persons[i]

for i in range(len(countries) - 1, -1, -1):
    if len(countries[i].persons) == 0:
        del countries[i]

In [8]:
jsonDump = 'rankings =' + os.linesep
jsonDump += json.dumps( {
                            'refreshed': refreshed,
                            'events': wcaEvents,
                            'persons': persons,
                            'countries': countries
                        },
                        default=lambda o: o.__dict__, ensure_ascii=False, indent=0)

with open(os.path.join('..', 'docs', 'data', 'Partial_Rankings.js'), 'w') as f:
    f.write(jsonDump)

In [9]:
print('Partial Rankings updated!')

Partial Rankings updated!


## All Done!